In [1]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori

In [3]:
sales_recipt = pd.read_csv("Dataset/201904 sales reciepts.csv")
sales_recipt.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [4]:
product = pd.read_csv("Dataset/product.csv")

In [5]:
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


#Data Wrangling

In [24]:
dataset = pd.merge(sales_recipt[['transaction_id','transaction_date', 'sales_outlet_id', 'customer_id', 'product_id', 'quantity']],
                      product[['product_id', 'product_category', 'product']],
                      on ='product_id', how = 'left' )
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


##Remove the sizes we only want to show the product

In [25]:
dataset[dataset['product'].str.contains("Dark chocolate")]['product'].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [26]:
dataset['product'].nunique()

80

In [27]:
# dataset['product'] = dataset['product'].str.replace(" Lg", "")
# dataset['product'] = dataset['product'].str.replace(" Rg", "")
# dataset['product'] = dataset['product'].str.replace(" Sm", "")
dataset['product']= dataset['product'].str.replace(' Rg','')
dataset['product']= dataset['product'].str.replace(' Sm','')
dataset['product']= dataset['product'].str.replace(' Lg','')

In [28]:
dataset['product'].nunique()

45

In [29]:
print(dataset['product'].unique())

['Traditional Blend Chai' 'Brazilian' 'Serenity Green Tea'
 'Our Old Time Diner Blend' 'Jamaican Coffee River' 'Ethiopia'
 'English Breakfast' 'Sustainably Grown Organic' 'Earl Grey' 'Cappuccino'
 'Espresso shot' 'Latte' 'Dark chocolate' 'Columbian Medium Roast'
 'Oatmeal Scone' 'Morning Sunrise Chai' 'Peppermint' 'Jumbo Savory Scone'
 'Lemon Grass' 'Chocolate Chip Biscotti' 'Spicy Eye Opener Chai'
 'Ginger Biscotti' 'Chocolate Croissant' 'Hazelnut Biscotti'
 'Cranberry Scone' 'Scottish Cream Scone ' 'Croissant' 'Almond Croissant'
 'Ginger Scone' 'Ouro Brasileiro shot' 'Organic Decaf Blend'
 'Chocolate syrup' 'Hazelnut syrup' 'Carmel syrup'
 'Sugar Free Vanilla syrup' 'Jamacian Coffee River'
 'Guatemalan Sustainably Grown' 'Civet Cat' 'Chili Mayan'
 'Primo Espresso Roast' 'Brazilian - Organic' 'I Need My Bean! Diner mug'
 'Espresso Roast' 'I Need My Bean! T-shirt' 'I Need My Bean! Latte cup']


Product subset;- exclude the tshirt and beans

In [30]:
products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]

In [31]:
dataset= dataset[dataset['product'].isin(products_to_take)]

In [33]:
dataset['product'].nunique()

18

In [34]:
dataset[['product','product_category']].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Latte,Coffee
3,Dark chocolate,Drinking Chocolate
4,Oatmeal Scone,Bakery
5,Jumbo Savory Scone,Bakery
6,Chocolate Chip Biscotti,Bakery
7,Ginger Biscotti,Bakery
8,Chocolate Croissant,Bakery
9,Hazelnut Biscotti,Bakery


In [36]:
dataset['transaction'] = dataset['transaction_id'].astype(str) + "_" + dataset['customer_id'].astype(str)

In [37]:
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino,127_116
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131


In [39]:
nums_of_items_for_each_transaction = dataset['transaction'].value_counts().reset_index()
nums_of_items_for_each_transaction

,transaction,count
0,209_0,31
1,206_0,30
2,204_0,27
3,208_0,25
4,203_0,24
...,...,...
8381,135_523,1
8382,130_157,1
8383,121_465,1
8384,118_748,1


In [42]:
nums_of_items_for_each_transaction[nums_of_items_for_each_transaction['count'] ==1]

,transaction,count
2646,471_8410,1
2647,1011_0,1
2648,341_5545,1
2649,1627_8289,1
2650,328_5635,1
...,...,...
8381,135_523,1
8382,130_157,1
8383,121_465,1
8384,118_748,1


In [44]:
valid_transactions = nums_of_items_for_each_transaction[nums_of_items_for_each_transaction['count']>1]['transaction'].tolist()
valid_transactions[:10]

['209_0',
 '206_0',
 '204_0',
 '208_0',
 '203_0',
 '207_0',
 '205_0',
 '151_0',
 '323_0',
 '301_0']

In [45]:
dataset = dataset[dataset['transaction'].isin(valid_transactions)]

In [46]:
dataset.shape

(10189, 9)

Product Trends

In [48]:
dataset['product_category'].value_counts()

product_category
Bakery                3800
Coffee                3174
Flavours              2246
Drinking Chocolate     947
Packaged Chocolate      22
Name: count, dtype: int64

In [47]:
dataset['product'].value_counts()

product
Cappuccino                  1290
Latte                       1256
Dark chocolate               969
Chocolate Croissant          636
Espresso shot                628
Sugar Free Vanilla syrup     605
Chocolate syrup              568
Carmel syrup                 561
Hazelnut syrup               512
Ginger Scone                 417
Jumbo Savory Scone           357
Croissant                    355
Chocolate Chip Biscotti      352
Cranberry Scone              350
Almond Croissant             347
Hazelnut Biscotti            338
Oatmeal Scone                334
Ginger Biscotti              314
Name: count, dtype: int64

##Popuarity engine

In [59]:
popularity_recommendation = dataset.groupby(['product','product_category']).count().reset_index()
popularity_recommendation

,product,product_category,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,transaction
0,Almond Croissant,Bakery,347,347,347,347,347,347,347
1,Cappuccino,Coffee,1290,1290,1290,1290,1290,1290,1290
2,Carmel syrup,Flavours,561,561,561,561,561,561,561
3,Chocolate Chip Biscotti,Bakery,352,352,352,352,352,352,352
4,Chocolate Croissant,Bakery,636,636,636,636,636,636,636
5,Chocolate syrup,Flavours,568,568,568,568,568,568,568
6,Cranberry Scone,Bakery,350,350,350,350,350,350,350
7,Croissant,Bakery,355,355,355,355,355,355,355
8,Dark chocolate,Drinking Chocolate,947,947,947,947,947,947,947
9,Dark chocolate,Packaged Chocolate,22,22,22,22,22,22,22


In [60]:
popularity_recommendation = popularity_recommendation[['product','product_category','transaction_id']]
popularity_recommendation = popularity_recommendation.rename(columns = {'transaction_id':'number_of_transactions'})

In [63]:
popularity_recommendation.head()

,product,product_category,number_of_transactions
0,Almond Croissant,Bakery,347
1,Cappuccino,Coffee,1290
2,Carmel syrup,Flavours,561
3,Chocolate Chip Biscotti,Bakery,352
4,Chocolate Croissant,Bakery,636


In [65]:
popularity_recommendation.to_csv('api/recommendation_objects/popularity_recommendation.csv',index=False)